# ANALYSE : LOCALISATIONS ET TAILLES DES ENTREPRISES EN FRANCE #

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, LabelSet
from bokeh.layouts import row, column
output_notebook()


Loading BokehJS ...

# IMPORT DES DATASETS #

In [2]:
#import du csv relatif aux établissements, fichier fourni dans le cadre du projet French Industry
ets = pd.read_csv('./data/base_etablissement_par_tranche_effectif.csv')

#import du csv relatif aux des données géographiques, fichier fourni dans le cadre du projet French Industry
geo = pd.read_csv('./data/name_geographic_information.csv')



# PREPARATION DES DATASETS #

In [3]:
#Renomme les colonnes du fichier des établissements pour une meilleure compréhension des variables de tailles (en salariés)
ets.columns = ['CODGEO', 'LIBGEO', 'REG', 'DEP', 'Total_ets','0/NA', '1 à 5', '6 à 9', '10 à 19', '20 à 49', '50 à 99','100 à 199', '200 à 499', '500 et +']

In [4]:
ets.head(1)

,CODGEO,LIBGEO,REG,DEP,Total_ets,0/NA,1 à 5,6 à 9,10 à 19,20 à 49,50 à 99,100 à 199,200 à 499,500 et +
0,01001,L'Abergement-Clémenciat,82,01,25,22,1,2,0,0,0,0,0,0


In [5]:
geo.head(1)

,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,numéro_circonscription,nom_commune,codes_postaux,code_insee,latitude,longitude,éloignement
0,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Attignat,01340,1024,46.283333,5.166667,1.21


**Le code_insee du fichier geo n'est pas au même format que le CODGEO du fichier ets.  
La variable du fichier geo doit également être renommée en CODGEO en lieu et place de code_insee.  
S'agissant d'un code, on préferera garder un format 'string' à un format numérique.  
Il manque de ce fait des 0 aux CODGEO du fichier geo ne présentant que 4 caractères.
Les CODGEO relatifs à la Corse peuvent poser problème. En effet, au lieu de commencer par 2A  
ou 2B, il semblent commencer par 20, ce qui était l'ancienne nomenclature. 
On doit donc les remplacer leur 0 par A ou B en fonction du département**

In [6]:
#Renomme la variable code_insee du fichier geo en CODGEO
geo = geo.rename(columns={'code_insee': 'CODGEO'})

In [7]:
#Passe la variable CODGEO au format String
geo.CODGEO = geo.CODGEO.astype('string')

In [8]:
#Création d'une fonction pour ajout d'un 0 si len de CODGEO == 4
geo['CODGEO'] = geo.CODGEO.apply(lambda x: x if len(x) == 5 else '0'+x)

In [9]:
#Gestion du problème Corse ;-)
#On doit donc remplacer par A ou B le 0 (deuxième caractère du code insee)
#Pour savoir si on doit remplacer par A ou B, il faut se référer à la variable numéro de département.
#On doit donc appliquer une fonction qui agit à la fois sur la variable numéro_département
#Mais aussi sur la variable CODGEO

#Création d'une fonction

def pb_corse(x):
    dpts_corse = ['2A', '2B']
    if x['numéro_département'] in dpts_corse:
        nouveau_code = x['numéro_département'] + x['CODGEO'][2:]
        return nouveau_code
    else:
        return x['CODGEO']

geo['CODGEO'] = geo.apply(lambda x: pb_corse(x), axis=1)

In [10]:
#Des doublons de CODGEO dans un des deux fichiers (ets et geo) ?
print(ets.CODGEO.duplicated().sum())
print(geo.CODGEO.duplicated().sum())

0
147


**On observe des doublons de CODGEO dans le fichier geo uniquement. Pourquoi?**

In [11]:
geo[geo.CODGEO.duplicated() == True].head(10).sort_values(by='CODGEO')

,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,numéro_circonscription,nom_commune,codes_postaux,CODGEO,latitude,longitude,éloignement
2293,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,06,Alpes-Maritimes,Nice,2,Nice,06000,06088,43.700000,7.25,1.08
2296,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,06,Alpes-Maritimes,Nice,3,Nice,06000,06088,43.700000,7.25,1.08
2361,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,06,Alpes-Maritimes,Nice,5,Nice,06000,06088,43.700000,7.25,1.08
2994,Est,21,Champagne-Ardenne,Châlons-en-Champagne,08,Ardennes,Charleville-Mézières,2,Charleville-Mézières,08000,08105,49.766667,4.716667,1.00
3909,Est,21,Champagne-Ardenne,Châlons-en-Champagne,10,Aube,Troyes,2,Troyes,10000,10387,48.300000,4.083333,0.34
4022,Est,21,Champagne-Ardenne,Châlons-en-Champagne,10,Aube,Troyes,3,Troyes,10000,10387,48.300000,4.083333,0.34
4771,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,13,Bouches-du-Rhône,Marseille,2,Marseille,13008 13006,13055,43.300000,5.4,NaN
4772,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,13,Bouches-du-Rhône,Marseille,3,Marseille,13001 13002 13007,13055,43.300000,5.4,NaN
4773,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,13,Bouches-du-Rhône,Marseille,4,Marseille,13003 13015 13016,13055,43.300000,5.4,NaN
4774,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,13,Bouches-du-Rhône,Marseille,5,Marseille,13005 13006 13010,13055,43.300000,5.4,NaN


**Il apparaît que les numéros de circonscriptions et les codes postaux sont la source de doublons**

**2 Solutions :**
- 1) Créér des clés uniques en concaténant CODGEO + numéro_circonscription + codes_postaux s'il on veut garder ces variables
- 2) Supprimer les doublons à partir de la variable CODGEO en acceptant de perdre des informations de circonscriptions et de codes postaux.

Dans un premier temps, on choisit la solution N°2

# JOINTURE DES DATASETS #

Quelles sont les colonnes du fichier geo que l'on veut garder ?
- **EU_circo** : <font color='green'>**oui**</font>, il s'agit des grandes régions ce qui peut être une variable intéressante pour l'analyse
- **code_région** : <font color='red'>**non**</font>, déjà présent dans ets
- **nom_région** : <font color='green'>**oui**</font>, absent du fichier ets et utile pour les visualisations
- **chef.lieu_région** : <font color='red'>**non**</font>, à voir peut-être plus tard
- **numéro_département** : <font color='red'>**non**</font>, déjà présent dans ets
- **préfecture** : <font color='red'>**non**</font>, à voir peut-être plus tard
- **numéro_circonscription** : <font color='red'>**non**</font>, notamment après le dédoublonage opéré précedemment
- **nom_commune** : <font color='red'>**non**</font>, déjà présent dans ets (LIBGEO)
- **codes_postaux** : <font color='red'>**non**</font>, à voir peut-être plus tard
- **CODGEO** : <font color='green'>**oui**</font>, nécessaire pour la jointure
- **latitude** : <font color='green'>**oui**</font>, peut servir pour les cartographies
- **longitude** : <font color='green'>**oui**</font>, peut servir pour les cartographies
- **éloignement** : <font color='red'>**non**</font>, à voir peut-être plus tard

In [12]:
#Suppression des doublons sur le CODGEO
geo = geo.drop_duplicates(subset='CODGEO').reset_index(drop=True)


In [13]:
#Supression des variables geo décidées comme non nécessaires
geo = geo.drop(['code_région', 'chef.lieu_région', 'numéro_département', 'préfecture', 'numéro_circonscription', 'nom_commune', 'codes_postaux', 'éloignement'], axis=1)


In [14]:
#Combien de lignes dans le fichier ets avant jointure ?
len(ets)

36681

In [15]:
#Jointure de méthode 'left' pour garder le fichier ets comme référence
ets_geo = ets.merge(geo, how='left', left_on='CODGEO', right_on='CODGEO')

In [16]:
#Combien de lignes dans le fichier ets_geo après jointure ?
len(ets_geo)

36681

La jointure disposant du même nombre de lignes que le fichier ets avant jointure, elle semble correcte.
Certaines lignes présentent cependant des NaN mais on en dénombre seulement 21 sur les variables géographiques principales (en exclusant longitude et latitude)

In [17]:
ets_geo.isna().sum()

CODGEO                0
LIBGEO                0
REG                   0
DEP                   0
Total_ets             0
0/NA                  0
1 à 5                 0
6 à 9                 0
10 à 19               0
20 à 49               0
50 à 99               0
100 à 199             0
200 à 499             0
500 et +              0
EU_circo             21
nom_région           21
nom_département      21
latitude           2934
longitude          2849
dtype: int64

Quels sont ces NaN ?

In [18]:
ets_geo[ets_geo['EU_circo'].isna()]

,CODGEO,LIBGEO,REG,DEP,Total_ets,0/NA,1 à 5,6 à 9,10 à 19,20 à 49,50 à 99,100 à 199,200 à 499,500 et +,EU_circo,nom_région,nom_département,latitude,longitude
9445,26020,La Répara-Auriples,82,26,24,22,2,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
12163,31300,Lieoux,73,31,2,2,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
14112,35317,Saint-Symphorien,53,35,15,6,4,2,2,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN
19971,52033,Avrecourt,21,52,4,3,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
20042,52124,Chézeaux,21,52,2,1,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
20156,52266,Laneuville-à-Rémy,21,52,4,2,2,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
20166,52278,Lavilleneuve-au-Roi,21,52,4,3,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
20313,52465,Saulxures,21,52,2,1,0,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
21270,55039,Beaumont-en-Verdunois,41,55,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
21281,55050,Bezonvaux,41,55,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


Les codes ci-dessus ne se trouvent pas dans le fichier geo, certainement parce que le fichier geo n'est pas très actuel. On les retrouvent en revanche dans le fichier ets.

In [19]:
#export des NaN concernant les infos régionales, départementales
geo_introuvables = ets_geo[ets_geo['EU_circo'].isna()].reset_index(drop=True)
geo_introuvables.to_csv('./NaN_geo/geo_introuvables.csv', index=False)
#Export des NaN latitude et longitude
gps_introuvables = ets_geo[ets_geo['latitude'].isna()].reset_index(drop=True)
gps_introuvables.to_csv('./NaN_geo/gps_introuvables.csv', index=False)

On supprime désormais les NaN (hors lattitude/longitude)

In [20]:
ets_geo = ets_geo.dropna(subset=['EU_circo']).reset_index(drop=True)

In [21]:
ets_geo.head()

,CODGEO,LIBGEO,REG,DEP,Total_ets,0/NA,1 à 5,6 à 9,10 à 19,20 à 49,50 à 99,100 à 199,200 à 499,500 et +,EU_circo,nom_région,nom_département,latitude,longitude
0,01001,L'Abergement-Clémenciat,82,01,25,22,1,2,0,0,0,0,0,0,Sud-Est,Rhône-Alpes,Ain,46.15,4.916667
1,01002,L'Abergement-de-Varey,82,01,10,9,1,0,0,0,0,0,0,0,Sud-Est,Rhône-Alpes,Ain,46.00,5.416667
2,01004,Ambérieu-en-Bugey,82,01,996,577,272,63,46,24,9,3,2,0,Sud-Est,Rhône-Alpes,Ain,45.95,5.35
3,01005,Ambérieux-en-Dombes,82,01,99,73,20,3,1,2,0,0,0,0,Sud-Est,Rhône-Alpes,Ain,46.00,4.9
4,01006,Ambléon,82,01,4,4,0,0,0,0,0,0,0,0,Sud-Est,Rhône-Alpes,Ain,45.75,5.6


In [22]:
len(ets_geo)

36660

# FONCTION #

In [23]:
grandesReg = ets_geo.groupby('EU_circo').Total_ets.sum()

In [24]:
grandesReg

EU_circo
Centre            265317
Est               460762
Nord-Ouest        454325
Ouest             502543
Outre-Mer         162375
Sud-Est           962581
Sud-Ouest         671292
Île-de-France    1048252
Name: Total_ets, dtype: int64

In [25]:

def ets_france():
    
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX        GRANDES REGIONS             XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX     NOMBRE D'ENTREPRISES           XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX    
   
    #Création d'un tableau
    grandesReg = ets_geo.groupby('EU_circo').agg({'Total_ets':'sum'}).sort_values(by='Total_ets', ascending=False)
    
    
    #Création d'une source
    src = ColumnDataSource({'Grandes régions': grandesReg.index, 
                            'Nombre d\'entreprises': grandesReg.Total_ets, 
                            'labels':["{:,.0f}K".format(i / 1000) for i in grandesReg.Total_ets]})
    
    #Création de la figure
    p1 = figure(plot_width = 300, 
               plot_height=400, 
               x_axis_label = 'Grandes régions', 
               y_axis_label = 'Nombre d\'entreprises',
               x_range = [i for i in grandesReg.index],
               y_range=(0, grandesReg.Total_ets.max() * 1.1),
               title='Total entreprises')
    
    #Optimisation du graphique
    #formatage des ordonnées
    p1.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p1.toolbar_location = None
    p1.title.align = 'center'
    p1.xaxis.major_label_orientation = np.pi/2
    
    #labels des barres
    s = LabelSet(x='Grandes régions', 
                 y='Nombre d\'entreprises', 
                 text='labels', 
                 source=src,
                 y_offset = 5,
                 text_align='center',
                 text_font_size="6pt")
    
    p1.add_layout(s)
    
    #Création du grahique
    p1.vbar(source=src, x='Grandes régions', 
           top='Nombre d\'entreprises', 
           width= 0.8, 
           #fill_color='firebrick', 
           line_color='black',
           line_width=0.3)

#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX        GRANDES REGIONS             XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  ENTREPRISES + DE 500 SALARIES     XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX   
    
    #Création d'un tableau
    Reg_taille_500 = ets_geo.groupby('EU_circo').agg({'500 et +':'sum'}).sort_values(by='500 et +', ascending=False)
    
    
    #Création d'une source
    src2 = ColumnDataSource({'Grandes régions': Reg_taille_500.index, 
                            'Nombre d\'entreprises': Reg_taille_500['500 et +'], 
                            'labels':["{:,.0f}".format(i) for i in Reg_taille_500['500 et +']]})
    
    #Création de la figure
    p2 = figure(plot_width = 300, 
               plot_height=400, 
               x_axis_label = 'Grandes régions', 
               y_axis_label = 'Nombre d\'entreprises',
               x_range = [i for i in Reg_taille_500.index],
               y_range=(0, Reg_taille_500['500 et +'].max() * 1.1),
               title='+ de 500 salariés')
    
    #Optimisation du graphique
    #formatage des ordonnées
    p2.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p2.toolbar_location = None
    p2.title.align = 'center'
    p2.xaxis.major_label_orientation = np.pi/2
    
    #labels des barres
    s2 = LabelSet(x='Grandes régions', 
                 y='Nombre d\'entreprises', 
                 text='labels', 
                 source=src2,
                 y_offset = 5,
                 text_align='center',
                 text_font_size="6pt")
    
    p2.add_layout(s2)
    
    #Création du grahique
    p2.vbar(source=src2, x='Grandes régions', 
           top='Nombre d\'entreprises', 
           width= 0.8, 
           fill_color='firebrick', 
           line_color='black',
           line_width=0.3)

#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX        GRANDES REGIONS             XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX            RATIOS                  XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX    
    
    
        #Création d'un tableau
    Ratio_total_500 = ets_geo.groupby('EU_circo').agg({'Total_ets':'sum', '500 et +':'sum'}).sort_values(by='Total_ets', ascending=False)
    Ratio_total_500['Ratio'] = Ratio_total_500['500 et +'] / Ratio_total_500['Total_ets']
    Ratio_total_500 = Ratio_total_500.sort_values(by='Ratio', ascending=True)
    
    #Création d'une source
    src3 = ColumnDataSource({'Grandes régions': Ratio_total_500.index, 
                            'Ratio': Ratio_total_500['Ratio'], 
                            'labels':["{:.3%}".format(i) for i in Ratio_total_500['Ratio']]})
    
    #Création de la figure
    p3 = figure(plot_width = 300, 
               plot_height=400, 
               x_axis_label = 'Ratios entreprises +500 salariés \n / total entreprises', 
               y_axis_label = 'Grandes régions',
               y_range = [i for i in Ratio_total_500.index],
               x_range=(0, Ratio_total_500['Ratio'].max() * 1.2),
               title='Ratios')
    
    #Optimisation du graphique
    #formatage des ordonnées
    p3.xaxis[0].formatter = NumeralTickFormatter(format='0.000%')
    p3.toolbar_location = None
    p3.title.align = 'center'
    p3.xaxis.major_label_orientation = np.pi/2
    
    #labels des barres
    s3 = LabelSet(x='Ratio', 
                 y='Grandes régions', 
                 text='labels', 
                 source=src3,
                 x_offset = 5,
                 y_offset = -5,
                 text_font_size="6pt")
    
    p3.add_layout(s3)
    
    #Création du grahique
    p3.hbar(source=src3, 
           right='Ratio', 
           y ='Grandes régions', 
           height= 0.8, 
           fill_color='green', 
           line_color='black',
           line_width=0.3)
    
    r = row(p1, p2, p3)
    
    #Visualisation de la figure
    show(r)


In [26]:
ets_france()